In [1]:
from pysat.formula import CNF
from pysat.solvers import Lingeling
import itertools
import math
import pickle

In [2]:
def construct_graph(solution, num_vertices):
    """
    given solution in the solver's format and number of vertices, it outputs a list of edges that can be passed
    into Sage to construct a graph"""
    vertices_lst = list(range(1, num_vertices+1))
    edge_lst = list(itertools.combinations(vertices_lst, 2))
    true_edge_lst = []
    for i in range(len(edge_lst)):
        edge = edge_lst[i]
        if solution[i] > 0:
            true_edge_lst.append(edge)
    return true_edge_lst

In [3]:
def save(edge_lst, file_name):
    """
    save the list of edges into a file
    """
    f = open(file_name, "w")
    for edges in edge_lst:
        f.write(str(edges) + '\n')
    f.close()

In [4]:
def solve_cnf(file, num_vertices):
    """
    The program input a DIMACS file and number of vertices, then solve for it recursively and output a list of
    edge list that Sage can take in. 
    We didn't use the default enumeration function of PySAT since instead of blocking add the variables, we only 
    want to block the variables that represent edges of the graph.
    """
    num_edges = int(math.factorial(num_vertices)/(math.factorial(2)*math.factorial(num_vertices-2)))
    cnf = CNF()
    cnf.from_file(file)
    s = Lingeling()
    solution_exist = True
    s.append_formula(cnf.clauses)
    count = 0
    edge_lst = []
    while solution_exist == True:
        solve = s.solve()
        solution = s.get_model()
        if solve == True:
            edges = construct_graph(solution[:num_edges], num_vertices)
            edge_lst.append(edges)
            count += 1
            print (count)
            assumption = [ -i for i in solution[:num_edges] ]
            s.add_clause(assumption)
        else:
            solution_exist == False
            break
    #save(edge_lst, "sample_"+str(num_vertices)+"_vertices") 
    return edge_lst

In [5]:
def solve_cnf_pickle(file, num_vertices):
    """
    similar to the above function, but instead of returning a list, we directly dump the solution into a file
    """
    num_edges = int(math.factorial(num_vertices)/(math.factorial(2)*math.factorial(num_vertices-2)))
    cnf = CNF()
    cnf.from_file(file)
    s = Lingeling()
    solution_exist = True
    s.append_formula(cnf.clauses)
    count = 0
    file = "candidates_"+str(num_vertices)+"v"
    outfile = open(file,'wb')
    while solution_exist == True:
        solve = s.solve()
        solution = s.get_model()
        if solve == True:
            edges = construct_graph(solution[:num_edges], num_vertices)
            pickle.dump(edges,outfile)
            count += 1
            if count % 10000 == 0:
                print (count)
            assumption = [ -i for i in solution[:num_edges] ]
            s.add_clause(assumption)
        else:
            solution_exist == False
            break
    print (str(num_vertices) + " has " + str(count) + " solutions")
    outfile.close()

In [ ]:
for i in range(15, 18):
    file = "all_constraints_" + str(i)
    solve_cnf_pickle(file, i)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000


In [2]:
def colorable(edge_lst):
    """
    given a list of edges, return a valid coloring if possible
    """
    solver = Lingeling()
    for edge in edge_lst:
        solver.add_clause(tuple([-edge[0],-edge[1]])) #no two adjacent vertices can be both 1
    vertices_lst = list(range(1, (max(max(edge_lst,key=lambda item:item[1])))+1))
    potential_triangles = list(itertools.combinations(vertices_lst, 3))
    for triangle in potential_triangles:
        v1 = triangle[0]
        v2 = triangle[1]
        v3 = triangle[2]
        if ([v1, v2] in edge_lst or [v2,v1] in edge_lst) and ([v2, v3] in edge_lst or [v3, v2] in edge_lst) and ([v1, v3] in edge_lst or [v3,v1] in edge_lst):
            solver.add_clause((v1,v2,v3))
            solver.add_clause((-v1, -v2))
            solver.add_clause((-v2, -v3))
            solver.add_clause((-v1, -v3))
        """
        if the triangle exists in this particular graph, it must satisfy 010 coloring
        """
    if solver.solve() == True:
        return solver.get_model()
    else:
        return False

In [3]:
edge_lst = [[1, 15],
 [1, 16],
 [1, 17],
 [1, 18],
 [2, 13],
 [2, 14],
 [2, 18],
 [3, 10],
 [3, 11],
 [3, 12],
 [3, 14],
 [3, 17],
 [4, 8],
 [4, 9],
 [4, 12],
 [4, 13],
 [5, 7],
 [5, 9],
 [5, 11],
 [5, 18],
 [6, 7],
 [6, 8],
 [6, 14],
 [6, 16],
 [7, 8],
 [7, 10],
 [7, 18],
 [8, 12],
 [9, 11],
 [9, 13],
 [10, 13],
 [10, 15],
 [11, 14],
 [12, 17],
 [13, 15],
 [14, 16],
 [15, 16],
 [17, 18]]
colorable(edge_lst)

False

In [ ]:
candidates = []
"""
open up the file with all candidates and append them into a list
"""
with (open("candidates_22v", "rb")) as openfile:
    while True:
        try:
            candidates.append(pickle.load(openfile))
        except EOFError:
            break

In [ ]:
for candidate in reversed(candidates):
    if colorable(candidate) == False:
        print (candidate)

In [ ]:
candidate_17 = [(1,8),(1,6),(1,13),(1,15),(2,9),(2,15),(2,10),(2,7),(3,14),(3,8),(3,16),(3,10),(4,14),(4,12),(4,11),(4,9),(5,11),(5,16),(5,17),(5,15),(6,12),(6,16),(6,13),(7,17),(7,10),(7,13),(8,14),(8,15),(9,12),(9,15),(10,16),(11,14),(11,17),(12,16),(13,17)]

In [ ]:
candidate_22_1 = [(1, 19), (1, 20), (1, 21), (1, 22), (2, 15), (2, 16), (2, 17), (2, 18), (3, 12), (3, 13), (3, 14), (3, 22), (4, 11), (4, 14), (4, 18), (4, 21), (5, 10), (5, 13), (5, 18), (5, 20), (6, 9), (6, 10), (6, 17), (6, 19), (7, 8), (7, 11), (7, 12), (8, 10), (8, 11), (8, 16), (9, 12), (9, 17), (9, 20), (10, 18), (10, 19), (11, 13), (11, 17), (11, 21), (12, 14), (12, 20), (13, 17), (13, 22), (14, 18), (15, 16), (15, 20), (16, 22), (19, 22), (20, 21)]

In [ ]:
colorable(candidate_22_1)

In [7]:
def num_of_subclause(file):
    """
    takes in a DIMACS file and output the number of subclauses with 2 variables
    """
    subclause_dict = {}
    cnf = CNF()
    cnf.from_file(file)
    clause_lst = cnf.clauses
    for clause in clause_lst:
        for pair in itertools.combinations(clause, 2):
            subclause = sorted(pair)
            try:
                subclause_dict[str(subclause)] += 1
            except:
                subclause_dict[str(subclause)] = 1
    return subclause_dict

In [9]:
num_of_subclause(r"script/all_constraints_10")

{'[-47, 51]': 1,
 '[-46, -1]': 1,
 '[-1, 51]': 1,
 '[-46, 51]': 1,
 '[-51, 47]': 2,
 '[-51, 1]': 1,
 '[1, 47]': 1,
 '[-51, 46]': 1,
 '[46, 47]': 1,
 '[-48, 52]': 1,
 '[-47, -1]': 1,
 '[-1, 52]': 1,
 '[-47, 52]': 1,
 '[-52, 48]': 2,
 '[-52, 1]': 1,
 '[1, 48]': 1,
 '[-52, 47]': 1,
 '[47, 48]': 1,
 '[-49, 53]': 1,
 '[-48, -1]': 1,
 '[-1, 53]': 1,
 '[-48, 53]': 1,
 '[-53, 49]': 2,
 '[-53, 1]': 1,
 '[1, 49]': 1,
 '[-53, 48]': 1,
 '[48, 49]': 1,
 '[-51, 55]': 1,
 '[-50, -2]': 1,
 '[-2, 55]': 1,
 '[-50, 55]': 1,
 '[-55, 51]': 2,
 '[-55, 2]': 1,
 '[2, 51]': 1,
 '[-55, 50]': 1,
 '[50, 51]': 1,
 '[-52, 56]': 1,
 '[-51, -2]': 1,
 '[-2, 56]': 1,
 '[-51, 56]': 1,
 '[-56, 52]': 2,
 '[-56, 2]': 1,
 '[2, 52]': 1,
 '[-56, 51]': 1,
 '[51, 52]': 1,
 '[-53, 57]': 1,
 '[-52, -2]': 1,
 '[-2, 57]': 1,
 '[-52, 57]': 1,
 '[-57, 53]': 2,
 '[-57, 2]': 1,
 '[2, 53]': 1,
 '[-57, 52]': 1,
 '[52, 53]': 1,
 '[-55, 59]': 1,
 '[-54, -3]': 1,
 '[-3, 59]': 1,
 '[-54, 59]': 1,
 '[-59, 55]': 2,
 '[-59, 3]': 1,
 '[3, 55]': 

In [19]:
"""
output in the format of "number of appearance: number of clauses"
"""
plot_dict = {}
subclause_dict = num_of_subclause(r"script/all_constraints_12")
for i in range(1, 1000):
    #print (str(i) + ": " +str(sum(x == i for x in subclause_dict.values())))
    plot_dict[i] = sum(x == i for x in subclause_dict.values())
print (plot_dict)

{1: 27126, 2: 2685, 3: 0, 4: 66, 5: 0, 6: 0, 7: 0, 8: 0, 9: 660, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0, 41: 0, 42: 0, 43: 0, 44: 0, 45: 0, 46: 0, 47: 0, 48: 0, 49: 0, 50: 0, 51: 0, 52: 0, 53: 0, 54: 0, 55: 0, 56: 0, 57: 0, 58: 0, 59: 0, 60: 0, 61: 0, 62: 0, 63: 0, 64: 0, 65: 0, 66: 0, 67: 0, 68: 0, 69: 0, 70: 0, 71: 0, 72: 0, 73: 0, 74: 0, 75: 0, 76: 0, 77: 0, 78: 0, 79: 0, 80: 0, 81: 0, 82: 0, 83: 0, 84: 0, 85: 0, 86: 0, 87: 0, 88: 0, 89: 0, 90: 0, 91: 0, 92: 0, 93: 0, 94: 0, 95: 0, 96: 0, 97: 0, 98: 0, 99: 0, 100: 0, 101: 0, 102: 0, 103: 0, 104: 0, 105: 0, 106: 0, 107: 0, 108: 0, 109: 0, 110: 0, 111: 0, 112: 0, 113: 0, 114: 0, 115: 0, 116: 0, 117: 0, 118: 0, 119: 0, 120: 0, 121: 0, 122: 0, 123: 0, 124: 0, 125: 0, 126: 0, 127: 0, 128: 0, 129: 0, 130: 0, 131: 0, 132: 0, 133: 0, 134: 0, 135: 0, 136: 0, 137: 0, 1

In [20]:
list(range(0,12))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [ ]:
from pysat.solvers import Lingeling
lst = []
cnf = CNF()
cnf.from_file("all_constraints_22_c2")
Solver = Lingeling
with Solver(bootstrap_with=cnf.clauses, use_timer=True) as s:
    print(s.solve())
    print('{0:.2f}s'.format(s.time()))
    print('{0:.2f}s'.format(s.time_accum()))